In [1]:
!pip install max --index-url https://dl.modular.com/public/nightly/python/simple/

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.8/284.8 MB 4.2 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/modular/mojo-gpu-puzzles

Cloning into 'mojo-gpu-puzzles'...
remote: Enumerating objects: 4059, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 4059 (delta 151), reused 127 (delta 127), pack-reused 3893 (from 2)
Receiving objects: 100% (4059/4059), 94.69 MiB | 41.81 MiB/s, done.
Resolving deltas: 100% (2489/2489), done.


In [3]:
!curl -fsSL https://astral.sh/uv/install.sh | sh

downloading uv 0.7.12 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [4]:
import max.support.notebook

In [5]:
def save_code_to_file(text: str, filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

In [18]:
mojo_code = """
from gpu import thread_idx, block_dim, block_idx
from gpu.host import DeviceContext
from layout import Layout, LayoutTensor
from testing import assert_equal

# ANCHOR: add_10_2d_layout_tensor
alias SIZE = 2
alias BLOCKS_PER_GRID = 1
alias THREADS_PER_BLOCK = (3, 3)
alias dtype = DType.float32
alias layout = Layout.row_major(SIZE, SIZE)


fn add_10_2d(
    output: LayoutTensor[mut=True, dtype, layout],
    a: LayoutTensor[mut=True, dtype, layout],
    size: Int,
):
    row = thread_idx.y
    col = thread_idx.x
    # FILL ME IN (roughly 2 lines)
    if row < size and col < size:
      output[row, col] = a[row, col] + 10


# ANCHOR_END: add_10_2d_layout_tensor


def main():
    with DeviceContext() as ctx:
        out_buf = ctx.enqueue_create_buffer[dtype](SIZE * SIZE).enqueue_fill(0)
        out_tensor = LayoutTensor[mut=True, dtype, layout](out_buf.unsafe_ptr())
        print("out shape:", out_tensor.shape[0](), "x", out_tensor.shape[1]())

        expected = ctx.enqueue_create_host_buffer[dtype](
            SIZE * SIZE
        ).enqueue_fill(0)

        a = ctx.enqueue_create_buffer[dtype](SIZE * SIZE).enqueue_fill(0)
        with a.map_to_host() as a_host:
            for i in range(SIZE * SIZE):
                a_host[i] = i
                expected[i] = a_host[i] + 10

        a_tensor = LayoutTensor[mut=True, dtype, layout](a.unsafe_ptr())

        ctx.enqueue_function[add_10_2d](
            out_tensor,
            a_tensor,
            SIZE,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        ctx.synchronize()

        with out_buf.map_to_host() as out_buf_host:
            print("out:", out_buf_host)
            print("expected:", expected)
            for i in range(SIZE * SIZE):
                assert_equal(out_buf_host[i], expected[i])

"""

In [19]:
save_code_to_file(mojo_code, "/content/mojo-gpu-puzzles/problems/p04/p04_layout_tensor.mojo")

In [20]:
!cd /content/mojo-gpu-puzzles && uv run poe p04_layout_tensor

Poe => mojo problems/p04/p04_layout_tensor.mojo
out shape: 2 x 2
out: HostBuffer([10.0, 11.0, 12.0, 13.0])
expected: HostBuffer([10.0, 11.0, 12.0, 13.0])
